## vcoabuarly 만들기

In [14]:
data_path = '../data/books_text_full/test/'

In [15]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

# 텍스트 파일의 내용을 변수 text로 리턴하는 함수
def load_doc(filename):
    # read only로 파일을 엽니다.
    file = open(filename, 'r', errors='replace')
    # 모든 텍스트를 읽습니다.
    text = file.read()
    # 파일을 닫습니다.
    file.close()
    return text

def clean_doc(doc):
    # white space 기준으로 tokenize 합니다.
    tokens = doc.split()
    # 각 token에서 모든 구두점을 삭제합니다.
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # 각 token에서 alaphabet으로만 이루어지지 않은 모든 단어를 삭제합니다.
    tokens = [word for word in tokens if word.isalpha()]
    # 각 token에서 stopwrods를 삭제합니다.
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # 각 token에서 1글자 이하인 모든 단어를 삭제합니다.
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

# 텍스트 파일을 불러와서 vocab에 추가하는 함수
def add_doc_to_vocab(filename, vocab):
    # 텍스트 파일을 불러옵니다.
    doc = load_doc(filename)
    # 텍스트 파일을 clean toekn으로 리턴합니다.
    tokens = clean_doc(doc)
    # clean token을 vocab에 추가합니다.
    vocab.update(tokens)

# 폴더에 있는 모든 문서를 vocab에 추가하는 함수
def process_docs(directory, vocab, is_train):
    # 폴더에 있는 모든 파일을 순회합니다.
    for filename in listdir(directory):
        # 인덱스가 새겨진 파일 이름과 is_train 인자를 기준으로 test set으로 분류할 모든 파일을 건너뜁니다.
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        # 폴더에 있는 파일의 절대 경로를 구합니다.
        path = directory + '/' + filename
        # 텍스트 파일을 불러와서 vocab에 추가하는 함수를 실행합니다.
        add_doc_to_vocab(path, vocab)

def save_list(lines, filename):
    # 각 문장을 하나의 텍스트 일부로 바꿉니다.
    data = '\n'.join(lines)
    # 파일을 쓰기 모드로 엽니다.
    file = open(filename, 'w')
    # 변환한 텍스트를 파일에 씁니다.
    file.write(data)
    # 파일을 닫습니다.
    file.close()

# vocab을 Counter() 객체로 할당합니다.
vocab = Counter()
# 폴더를 지정하고 폴더 내 모든 문서를 vocab에 추가합니다.
process_docs(data_path, vocab, True)
# vocab의 크기를 출력합니다.
print(len(vocab))
# vocab에서 가장 많이 등장한 50개 단어를 출력합니다.
print(vocab.most_common(50))

# token을 min_occurence 기준으로 유지합니다.
min_occurence = 1
tokens = [k for k,c in vocab.items() if c >= min_occurence]
print(len(tokens))
# token을 vocab 파일로 저장합니다.
save_list(tokens, 'corpusToLines_vocab.txt')
print("\n# 단어 {}개의 [corpusToLines_vocab.txt]로 저장했습니다.".format(len(tokens)))

# 보카를 불러옵니다.
vocab_filename = 'corpusToLines_vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print("# 단어 {}개의 [{}]을 [vocab]으로 불러왔습니다.".format(len(vocab), vocab_filename))

13538
[('Tick', 2045), ('He', 1384), ('The', 1296), ('said', 1071), ('like', 1003), ('Paul', 881), ('back', 777), ('Sofia', 730), ('one', 687), ('George', 670), ('Master', 573), ('could', 562), ('Jane', 527), ('But', 512), ('Sato', 495), ('looked', 483), ('know', 480), ('It', 475), ('Chu', 461), ('time', 455), ('didnt', 449), ('And', 445), ('eyes', 441), ('right', 429), ('She', 426), ('something', 423), ('hed', 403), ('man', 393), ('felt', 380), ('What', 376), ('around', 374), ('asked', 364), ('see', 360), ('Im', 353), ('away', 346), ('Mothball', 339), ('face', 334), ('get', 331), ('air', 327), ('Rutger', 324), ('would', 312), ('thought', 309), ('made', 308), ('us', 307), ('head', 307), ('You', 302), ('way', 296), ('thing', 277), ('things', 276), ('think', 273)]
13538

# 단어 13538개의 [corpusToLines_vocab.txt]로 저장했습니다.
# 단어 13538개의 [corpusToLines_vocab.txt]을 [vocab]으로 불러왔습니다.


## 전체 코퍼스를 문장 단위 리스트로 바꾸기

In [18]:
from string import punctuation
from os import listdir
from gensim.models import Word2Vec

# 텍스트 파일의 내용을 변수 text로 리턴하는 함수
def load_doc(filename):
    # read only로 파일을 엽니다.
    file = open(filename, 'r', errors='replace')
    # 모든 텍스트를 읽습니다.
    text = file.read()
    # 파일을 닫습니다.
    file.close()
    return text


# 문서를 cleaned lines(=tokens)으로 리턴합니다.
# tokens = line, token = word
def doc_to_clean_lines(doc, vocab):
    clean_lines = []
    lines = doc.split(".")
    for line in lines:
        # 공백 기준으로 token을 나눕니다.
        tokens = line.split()
        # 구두점 제거
        table = str.maketrans('', '', punctuation)
        tokens = [w.translate(table) for w in tokens]
        # vocab에 있는 단어만 추출합니다.
        tokens = [w for w in tokens if w in vocab]
        clean_lines.append(tokens)
    return clean_lines


# 폴더에 있는 모든 문서를 vocab에 추가하는 함수
def process_docs(directory, vocab, is_train):
    lines = list()
    # 폴더에 있는 모든 파일을 순회합니다.
    for filename in listdir(directory):
        # 폴더에 있는 파일의 절대 경로를 구합니다.
        path = directory + '/' + filename
        # 파일을 불러옵니다.
        doc = load_doc(path)
        # 파일을 개끗하게 만들어서 token으로 리턴합니다.
        doc_lines = doc_to_clean_lines(doc, vocab)
        # 리스트 객체에 추가합니다.
        lines += doc_lines
    return lines

# 보카를 불러옵니다.
vocab_filename = 'fantasy_vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# 모든 training set을 불러옵니다.
train_docs = process_docs(data_path, vocab, True)
sentences = train_docs
print('Total training sentences: %d' % len(sentences))

Total training sentences: 19160


## Word2Vec 만들기

In [19]:
from string import punctuation
from os import listdir
from gensim.models import Word2Vec

# 텍스트 파일의 내용을 변수 text로 리턴하는 함수
def load_doc(filename):
    # read only로 파일을 엽니다.
    file = open(filename, 'r', errors='replace')
    # 모든 텍스트를 읽습니다.
    text = file.read()
    # 파일을 닫습니다.
    file.close()
    return text


# 문서를 cleaned lines(=tokens)으로 리턴합니다.
# tokens = line, token = word
def doc_to_clean_lines(doc, vocab):
    clean_lines = []
    lines = doc.splitlines()
    for line in lines:
        # 공백 기준으로 token을 나눕니다.
        tokens = line.split()
        # 구두점 제거
        table = str.maketrans('', '', punctuation)
        tokens = [w.translate(table) for w in tokens]
        # vocab에 있는 단어만 추출합니다.
        tokens = [w for w in tokens if w in vocab]
        clean_lines.append(tokens)
    return clean_lines


# 폴더에 있는 모든 문서를 vocab에 추가하는 함수
def process_docs(directory, vocab, is_train):
    lines = list()
    # 폴더에 있는 모든 파일을 순회합니다.
    i = 0
    for filename in listdir(directory):
        # 인덱스가 새겨진 파일 이름과 is_train 인자를 기준으로 test set으로 분류할 모든 파일을 건너뜁니다.
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        # 폴더에 있는 파일의 절대 경로를 구합니다.
        path = directory + '/' + filename
        # 파일을 불러옵니다.
        doc = load_doc(path)
        # 파일을 개끗하게 만들어서 token으로 리턴합니다.
        doc_lines = doc_to_clean_lines(doc, vocab)
        # 리스트 객체에 추가합니다.
        lines += doc_lines
        i += 1
        if i % 300 == 0:
            print(lines)
    return lines

# 보카를 불러옵니다.
vocab_filename = 'fantasy_vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# 모든 training set을 불러옵니다.
train_docs = process_docs(data_path, vocab, True)
sentences = train_docs
print('Total training sentences: %d' % len(sentences))

# word2vec 모델을 훈련시킵니다.
model = Word2Vec(sentences, size=100, window=5, workers=8, min_count=1)
# 모델의 vocabulary size를 요약합니다.
words = list(model.wv.vocab)
print("Vocabulary size: %d" % len(words))

# 모델을 ASCII 포맷으로 저장합니다.
filename = 'fantasy_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

Total training sentences: 15240


Vocabulary size: 12674


In [25]:
type(sentences)
sentences[4:7]

[['It', 'soulikens'],
 [],
 ['Master',
  'George',
  'sat',
  'study',
  'lights',
  'dimmed',
  'Muffintops',
  'purring',
  'corner',
  'first',
  'light',
  'dawns',
  'birth',
  'still',
  'hour',
  'He',
  'stared',
  'wall',
  'fascinating',
  'thing',
  'Realities',
  'stapled',
  'see',
  'whenever',
  'wished',
  'knot',
  'wood',
  'paneling',
  'knot',
  'two',
  'eyes',
  'mouth',
  'looked',
  'right',
  'reason',
  'reminded',
  'boy',
  'named',
  'Atticus',
  'Higginbottom']]

## Pre-trained Word2vec 사용하기

In [39]:
from string import punctuation
from os import listdir
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding, Dropout, GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# 텍스트 파일의 내용을 변수 text로 리턴하는 함수
def load_doc(filename):
    # read only로 파일을 엽니다.
    file = open(filename, 'r', errors='replace')
    # 모든 텍스트를 읽습니다.
    text = file.read()
    # 파일을 닫습니다.
    file.close()
    return text


# 텍스트 파일을 clean token으로 리턴하는 함수
def clean_doc(doc, vocab):
    # white space 기준으로 tokenize 합니다.
    tokens = doc.split()
    # 각 token에서 모든 구두점을 삭제합니다.
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # 각 token에서 보카에 없는 단어는 걸러냅니다.
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens


# 폴더에 있는 모든 문서를 vocab에 추가하는 함수
def process_docs(directory, vocab, is_train):
    documents = list()
    # 폴더에 있는 모든 파일을 순회합니다.
    for filename in listdir(directory):
        # 인덱스가 새겨진 파일 이름과 is_train 인자를 기준으로 test set으로 분류할 모든 파일을 건너뜁니다.
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        # 폴더에 있는 파일의 절대 경로를 구합니다.
        path = directory + '/' + filename
        # 파일을 불러옵니다.
        doc = load_doc(path)
        # 파일을 개끗하게 만들어서 token으로 리턴합니다.
        tokens = clean_doc(doc, vocab)
        # 리스트 객체에 추가합니다.
        documents.append(tokens)
    return documents


# embedding을 dictionary로 불러옵니다.
def load_embedding(filename):
    # embedding을 메모리에 올려두되 첫번째 줄은 생략합니다.
    file = open(filename, 'r')
    lines = file.readlines()[1:]
    file.close()
    # 단어와 벡터를 연결하는 map을 생성합니다.
    embedding = {}
    for line in lines:
        parts = line.split()
        # 딕셔너리의 key는 단어에 대한 문자열이고, value는 벡터에 대한 numpy array입니다.
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding


# 불러온 embedding을 기준으로 Embedding layer의 weight matrix를 생성합니다.
def get_weight_matrix(embedding, vocab):
    # 전체 vocabulary 크기에 unknown words에 대한 숫자 0을 추가합니다.
    vocab_size = len(vocab) + 1
    # (vocab_size, 100) 만큼 weight matrix의 모든 값을 0으로 초기화해서 생성합니다.
    weight_matrix = zeros((vocab_size, 100))
    # Tokenizer의 정수 맵핑을 이옹하여 vocab의 모든 단어에 해당하는 벡터를 저장합니다.
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix


# 보카를 불러옵니다.
vocab_filename = 'fantasy_vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# 모든 training set을 불러옵니다.
train_docs = process_docs(data_path, vocab, True)
# 텍스트 문서를 정수로 맵핑시키는 tokenizer를 정의합니다.
tokenizer = Tokenizer()
# tokenizer를 모든 training 문서에 적용합니다.
tokenizer.fit_on_texts(train_docs)
# 텍스트 문서를 정수의 나열로 인코드합니다.
encoded_docs = tokenizer.texts_to_sequences(train_docs)
# 가장 길이가 긴 문서의 길이를 구하고, 그에 맞춰 패딩합니다.
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

Using TensorFlow backend.


In [62]:
# vocab_size를 정의합니다.
vocab_size = len(tokenizer.word_index) + 1
print("vocab_size:{}".format(vocab_size))

vocab_size:180036


In [64]:
len(embedding_vectors)

180036

In [65]:
print(max_length)

271436


## 모델 구성하기 (filter_size = [3, 4, 5])

In [60]:
from keras.layers import Embedding, Dropout, GlobalMaxPooling1D
from keras.layers import Merge
import keras.backend.tensorflow_backend as K

# 파일에서 embedding을 불러옵니다.
raw_embedding = load_embedding('fantasy_embedding_word2vec.txt')
# 벡터를 올바른 순서로 정렬합니다.
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# embedding layer를 만듭니다.
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

model = Sequential()

with K.tf.device('/gpu:0'):
    filter_sizes = [3,4,5]
    drop_out_rate = 0.5
    hidden_dims = 50
    
    model.add(embedding_layer)
    for idx, sz in enumerate(filter_sizes):
        model.add(Conv1D(filters=100, kernel_size=sz, activation='relu', strides=1, padding='valid'))
        model.add((MaxPooling1D(pool_size=2)))
        
    
#     model.add(Dropout(drop_out_rate))  
    model.add(Flatten())
#     model.add(Dense(hidden_dims, activation="relu"))
#     model.add(GlobalMaxPooling1D())
    print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 271436, 100)       18003600  
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 271434, 100)       30100     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 135717, 100)       0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 135714, 100)       40100     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 67857, 100)        0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 67853, 100)        50100     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 33926, 100)        0         
__________

---

In [52]:
# # Xtrain으로 3개씩 pair 만들기
# pair = [[Xtrain[i], Xtrain[i+1], Xtrain[i+2]] for i in range(len(Xtrain)) if i < len(Xtrain)-2]
# assert len(Xtrain) - 2 == len(pair)

In [53]:
import _pickle as cPickle
with open('pair.pkl', 'wb') as f:
    cPickle.dump(pair, f)

In [54]:
# print(pair[0], pair[1], sep='\n')

---

In [ ]:
# # 모델 정의
# model = Sequential()
# model.add(embedding_layer)
# model.add(Conv1D(filters=200, kernel_size=5, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))
# print(model.summary)

# # network를 컴파일합니다.
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# # network를 training data에 fit 합니다.
# model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# # 평가
# loss, acc = model.evaluate(Xtest, ytest, verbose=0)
# print('Test Accuracy: %f' % (acc*100))

---

pair 나누기

+ 최종 모델의 출력을 리스트로 출력하기

In [50]:
# CNN input data 묶기 (0, 1, 2), (1, 2, 3), (2, 3, 4)
def pairing(input2):
    newinput=[]
    for i in range(len(input2)-2):
        holdlist=[]
        newinput.append(holdlist)
        holdlist.append(input2[i])
        holdlist.append(input2[i+1])
        holdlist.append(input2[i+2])
    return newinput
            

# cosine similarity
import math
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)


# (0, 2):1, (1, 3):2, (2, 4):3
# 이 중에 왼쪽 pair cosine similarity
def side_cosine_similarity(v1_set, v2_set):
    first=cosine_similarity(v1_set[0], v2_set[0])
    second=cosine_similarity(v1_set[2], v2_set[2])
    
    return (first+second)/2


def center_cosine_similarity(v1_set, v2_set):
    return cosine_similarity(v1_set[1], v2_set[1])


def loss_sum(data_array):
    pairs = pairing(data_array)
    sum = 0
    for n1 in range(len(pairs)):
        for n2 in range(n1, len(pairs)):
            if not n1 == n2:
                loss=side_cosine_similarity(pairs[n1], pairs[n2])-center_cosine_similarity(pairs[n1], pairs[n2])
                sum+=loss**2
    return sum

model_output
loss_sum(model_output)

for i in range(len(pairs)):
    